To understand Decorators, lets first see

## What problem do decorators solve?

Imagine we have many functions like this:

In [1]:
def add(a, b):
    return a + b

def multiply(a, b):
    return a * b


Now we want to add extra behavior to these functions, like:
- Logging (print when a function is called)
- Timing (how long it takes)
- Authentication / permission checks

#### Bad way to do this: 
- copy-paste the same code inside every function

#### Best practice:
- write the extra behavior once and reuse it. **Decorators exist to do this cleanly.**

### Core idea:
    A decorator is a function that takes another function and returns a new function with extra behavior.

#### A Python fact:

In python, **functions are just objects**. This idea is the foundation of decorators. That means:
- We can pass a function to another function.
- We can return a function from a function.

For example:

In [40]:
def say_hello():
    print("Hello!")

x = say_hello   # no parentheses!
x()             # calls say_hello


Hello!


### Real-life analogy: Gift wrapping
- **Gift** is the `original function`.
- **Gift wrapper** is the `decorator`.
- **Wrapped gift** is the `new function with extra behaviour`.

## First simple decorator
#### Step 1: a normal function

In [41]:
def greet():
    print("Hello!")

#### Step 2: a function that adds extra behavior

In [42]:
def my_decorator(func):
    def wrapper():
        print("Before the function runs")
        func()
        print("After the function runs")
    return wrapper


Lets break this down:
- `func`: the function being decorated.
- `wrapper()`: New function that:
    - does something before.
    - calls the original function.
    - does somthing after.
- `return wrapper`: Replaces the original function.

#### Step 3: manually decorate the function

In [43]:
decorated_greet = my_decorator(greet)
decorated_greet()

Before the function runs
Hello!
After the function runs


`greet()` is now wrapped with extra behavior.

## The `@decorator` syntax (same thing, cleaner)

Instead of this:

In [44]:
greet = my_decorator(greet)

Python lets you write:

In [45]:
@my_decorator
def greet():
    print("Hello!")


This is just syntax sugar, same as `greet = my_decorator(greet)`

### Decorator with arguments

Most real functions take arguments.

In [46]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before function")
        result = func(*args, **kwargs)
        print("After function")
        return result
    return wrapper

Why `*args, **kwargs`?
- Works with any number of arguments
- Makes decorator reusable

In [47]:
@my_decorator
def add(a, b):
    return a + b

print(add(2, 3))

Before function
After function
5


### Real-world example 1: Timing a function

In [48]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Time taken: {end - start:.4f} seconds")
        return result
    return wrapper


In [49]:
@timer
def slow_function():
    time.sleep(1)

slow_function()


Time taken: 1.0020 seconds


- We added timing without touching the function code.

### Real-world example 2: Decorators with FastAPI

In [50]:
# If needed, uncomment these:
# !pip -q install fastapi uvicorn nest_asyncio

In [37]:
from fastapi import FastAPI
from contextlib import asynccontextmanager
import uvicorn

@asynccontextmanager
async def lifespan(app: FastAPI):
    print("App is starting up")
    yield
    print("App is shutting down")

app = FastAPI(lifespan=lifespan)

@app.get("/hello")
def hello():
    return {"message": "Hello from FastAPI!"}

# Jupyter-friendly: start server using await (no asyncio.run)
config = uvicorn.Config(app, host="127.0.0.1", port=8000, log_level="info")
server = uvicorn.Server(config)

await server.serve()


INFO:     Started server process [5364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


App is starting up


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5364]


App is shutting down


---